In [ ]:
import torch
from torch import nn
from torch.nn import Parameter
from torch.autograd import Variable
import numpy as np
import random

l_miast = 9
l_miesiecy = 12
factors = 3

months = ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
cities = ["Toronto", "Warsaw", "Boston", "London", "San Francisco", "Jerusalem", "Mexico", "Cape Town", "Sydney"]
avg_temp = np.array([
    [-5.8, -3.1, 4.5, 6.7, 14.3, 18.2, 20.1, 20.6, 15.9, 11.2, 3.6, -7.2],
    [-2.9, 3.6, 4.2, 9.7, 16.1, 19.5, 20.0, 18.8, 16.4, 7.6, 3.2, 1.3],
    [0.3, 1.5, 5.9, 8.4, 14.8, 20.2, 24.5, 24.7, 19.7, 13.0, 7.9, 1.9],
    [2.3, 6.5, 8.7, 9.2, 12.3, 15.4, 17.3, 20.0, 14.8, 10.8, 8.7, 6.4],
    [11.5, 13.9, 14.3, 15.7, 16.3, 17.4, 17.2, 17.7, 18.2, 17.4, 14.6, 10.4],
    [9.7, 10.3, 12.7, 15.5, 21.2, 22.1, 24.1, 25.3, 23.5, 20.1, 15.7, 11.8],
    [14.0, 15.6, 17.5, 20.3, 20.6, 18.1, 17.6, 18.2, 17.8, 16.8, 14.9, 16.0],
    [23.1, 23.3, 21.4, 19.0, 17.1, 15.5, 15.4, 15.6, 15.4, 18.6, 20.9, 21.3],
    [23.8, 24.6, 23.4, 20.8, 18.1, 15.1, 14.4, 14.5, 17.3, 19.0, 21.8, 24.3]
])

In [ ]:
train_data = torch.zeros(l_miast, l_miesiecy)

def wylosuj():
	for i in range(l_miast):
		t = random.sample(range(0, l_miesiecy-1), (l_miesiecy+1)/2)
		for j in t:
			train_data[i][j] = avg_temp[i][j]
		
		

class Factorize(nn.Module):
	def __init__(self):
        	super(Factorize, self).__init__()
        	self.cities_factors = Parameter(torch.Tensor(l_miast, factors))
        	self.months_factors = Parameter(torch.Tensor(l_miesiecy, factors))
        def forward(self):
        	return torch.mm(self.cities_factors, self.months_factors.transpose(0,1))


def loss_func(output):
	loss = 0
	for i in range(l_miast):
		for j in range(l_miesiecy):
			if train_data[i][j] != 0:
				loss = loss+(train_data[i][j]-output[i][j])*(train_data[i][j]-output[i][j])
	return loss		

def loss_test(pred):
	loss = 0
	for i in range(l_miast):
		for j in range(l_miesiecy):
			if train_data[i][j] == 0:
				loss = loss+(avg_temp[i][j]-pred[i][j])*(avg_temp[i][j]-pred[i][j])
	return loss

model = Factorize()
optimizer = torch.optim.SGD(list(model.parameters()), lr=1e-3)

In [ ]:
wylosuj()

for i in range(1):
	optimizer.zero_grad()
	output = model()
	loss = loss_func(output)
	print loss, i
	loss.backward()
	optimizer.step()
pred = model().detach().numpy()
print loss_test(pred)
